# Загружаем данные

In [1]:
import pandas as pd
from sklearn.utils import shuffle
twenty_train = pd.read_csv('train.csv', sep=',', encoding='utf-8')

Test = pd.read_csv('test.csv', sep=',', encoding='utf-8')
Test=Test[' Text']

# SGDClassifier

In [60]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
import numpy as np
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import RidgeClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.extmath import density
from sklearn import metrics

from sklearn import ensemble

text_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1, 6),analyzer='char_wb')),#ngram_range=(3, 6),max_df=0.5, analyzer='char' 0.8418936775195723 +/- 0.008995447490375489
                     ('tfidf', TfidfTransformer()), 
                     #('clf',LinearSVC(penalty="l2",loss='squared_hinge')),
                     ('clf', SGDClassifier(loss='modified_huber', random_state=42,max_iter=200)),
                     #('clf',PassiveAggressiveClassifier(C=0.5602439092715068)),
                      #('clf', MultinomialNB(alpha=0.33921084703630489,fit_prior=False)),
                      #  ('clf',BernoulliNB(alpha=0.13420609458144628)),#0.8429193534216901 +/- 0.009888828863667175
                     #('clf',RandomForestClassifier(n_estimators=100)),#0.7234268982515644 +/- 0.01047402145804584
                     
])

text_clf.fit(twenty_train.Text, twenty_train.Label)  

predicted = text_clf.predict(Test) 

score = cross_val_score(text_clf, twenty_train.Text, twenty_train.Label, scoring='accuracy', cv=8, n_jobs=-1)

print("{} +/- {}".format(score.mean(), score.std()))

predicted_train1 = text_clf.predict(twenty_train.Text)
np.mean(predicted_train1 == twenty_train.Label)  

0.8543824211501703 +/- 0.006707519728383528


0.96881403688524592


maxdf 05 0.8539984622177896 +/- 0.0074228040954732386
0.97060706967213117


0.8543181212463643 +/- 0.007143552490318967
0.96849385245901642



0.8435585739194156 +/- 0.007756685897387785



Out[89]:

0.99314805327868849


0.8413165596269867 +/- 0.007451230540229675

0.8418936775195723 +/- 0.008995447490375489
0.95600665983606559


# Порпробуем bagging

In [39]:
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
import numpy as np
from sklearn.feature_extraction.text import TfidfTransformer
#text_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1, 1))),
 #                    ('tfidf', TfidfTransformer(use_idf=True)),
  #                   ('clf', SGDClassifier(loss='squared_hinge', penalty='elasticnet',
  #                                         alpha=0.001, l1_ratio=0,random_state=42)),
#])
from sklearn.ensemble import BaggingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
tfidf_transformer = TfidfTransformer(use_idf=True,norm='l2')
count_vect = CountVectorizer(ngram_range=(1, 1),max_df=0.95,min_df=0.5, analyzer='word')
X_t= count_vect.fit_transform(twenty_train.Text)
X_train_tfidf = tfidf_transformer.fit_transform(X_t)
text_clf=BernoulliNB(alpha=0.13420609458144628)
bagging = BaggingClassifier(text_clf,n_estimators=100)
bagging.fit(X_train_tfidf, twenty_train.Label)  

X_new_counts = count_vect.transform(Test)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

predicted = bagging.predict(X_new_tfidf)     

score = cross_val_score(bagging, X_train_tfidf, twenty_train.Label, scoring='accuracy', cv=8, n_jobs=-1)

print("{} +/- {}".format(score.mean(), score.std()))

np.mean(predicted_train1 == twenty_train.Label)  

ValueError: After pruning, no terms remain. Try a lower min_df or a higher max_df.






0.8395897617311174 +/- 0.011688974709492388


0.8392695444822916 +/- 0.011801866273057224
0.95306096311475408

0.8303053628048254 +/- 0.013890627835110471
0.93205686475409832


In [119]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
import numpy as np
from sklearn.feature_extraction.text import TfidfTransformer
#text_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1, 1))),
 #                    ('tfidf', TfidfTransformer(use_idf=True)),
  #                   ('clf', SGDClassifier(loss='squared_hinge', penalty='elasticnet',
  #                                         alpha=0.001, l1_ratio=0,random_state=42)),
#])
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score

tfidf_transformer = TfidfTransformer(use_idf=True,norm='l2')
count_vect = CountVectorizer(ngram_range=(1, 1),max_df=0.5, analyzer='word')
clf= MultinomialNB(alpha=0.33921084703630489,fit_prior=False)
X_t= count_vect.fit_transform(twenty_train.Text)
X_train_tfidf = tfidf_transformer.fit_transform(X_t)
text_clf=BernoulliNB(alpha=0.13420609458144628)
clf2=SGDClassifier(loss='log', penalty='elasticnet',alpha=0.001, l1_ratio=0,random_state=42,n_iter=100)
eclf = VotingClassifier(estimators=[('dt', clf), ('knn', text_clf),('ss', clf2)], voting='hard')
eclf = eclf.fit( X_train_tfidf, twenty_train.Label)

X_new_counts = count_vect.transform(Test)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

predicted = eclf.predict(X_new_tfidf)     

score = cross_val_score(eclf, X_train_tfidf, twenty_train.Label, scoring='accuracy', cv=8, n_jobs=-1)

print("{} +/- {}".format(score.mean(), score.std()))

np.mean(predicted_train1 == twenty_train.Label)  


E:\install\anaconda\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:73: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


0.8415751029504578 +/- 0.009808439506498266


0.95043545081967218

# Подбор параметров через GridSearchCV

In [44]:
text_clf.get_params().keys()

dict_keys(['memory', 'steps', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__alpha', 'clf__average', 'clf__class_weight', 'clf__epsilon', 'clf__eta0', 'clf__fit_intercept', 'clf__l1_ratio', 'clf__learning_rate', 'clf__loss', 'clf__max_iter', 'clf__n_iter', 'clf__n_jobs', 'clf__penalty', 'clf__power_t', 'clf__random_state', 'clf__shuffle', 'clf__tol', 'clf__verbose', 'clf__warm_start'])

In [33]:
from sklearn.model_selection import GridSearchCV
parameters = {
    'vect__max_df': (0.9,0.91),
    'vect__min_df': (0.105,0.101),
    #'vect__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
    #'tfidf__norm': (None,'l1','l2'),
    #'clf__loss' :  ('hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron','squared_loss', 'huber', 'epsilon_insensitive','squared_epsilon_insensitive')
}

gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)

In [34]:
gs_clf = gs_clf.fit(twenty_train.Text, twenty_train.Label)

In [35]:
gs_clf.best_score_                                  
for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))

vect__max_df: 0.9
vect__min_df: 0.101


# Подбор параметров через RandomezedSearchCV

In [43]:

from scipy.stats import uniform as sp_rand
from sklearn.linear_model import Ridge
from sklearn.grid_search import RandomizedSearchCV
# prepare a uniform distribution to sample for the alpha parameter
parameters = {
             'vect__max_df': sp_rand()}
# create and fit a ridge regression model, testing random alpha valuesmodel = Ridge()
rsearch = RandomizedSearchCV(estimator=text_clf, param_distributions=parameters, n_iter=100)
rsearch.fit(twenty_train.Text, twenty_train.Label)


RandomizedSearchCV(cv=None, error_score='raise',
          estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='char_wb', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.5, max_features=None, min_df=1,
        ngram_range=(1, 6), preprocessor=None, stop_words=None,
        st...ty='l2', power_t=0.5, random_state=42,
       shuffle=True, tol=None, verbose=0, warm_start=False))]),
          fit_params={}, iid=True, n_iter=100, n_jobs=1,
          param_distributions={'vect__max_df': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000006660CF0F60>},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          scoring=None, verbose=0)

In [44]:
for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, rsearch.best_params_[param_name]))

vect__max_df: 0.87569318668869223


# Выведем результат в файл

In [62]:
df = pd.DataFrame(predicted,columns=['Prediction'])
df.index += 1 
df.to_csv('pred.csv',  index_label='Id', encoding='utf-8')